https://colab.research.google.com/drive/16q3uiwhQQoYC97izjzBE70-LLzvnOdVu#scrollTo=LHu0SMdSGqd8
https://androidkt.com/keras-custom-training/


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.datasets import cifar10
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dense, Activation ,AveragePooling2D,GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, auc,precision_score, recall_score,f1_score,accuracy_score
from sklearn.model_selection import StratifiedKFold
from scipy import interp
import matplotlib.pyplot as plt
from skimage import transform
import scipy.ndimage
import matplotlib.pyplot as plt

from keras.layers import Lambda, Input
from keras.models import Model, Sequential
import tensorflow as ktf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [ ]:
BATCH_SIZE=32
IMG_SIZE=32

In [ ]:
import tensorflow_datasets as tfds
train_ds = tfds.load(name="cifar10",split=tfds.Split.TRAIN)
test_ds = tfds.load(name="cifar10",split=tfds.Split.TEST)

In [ ]:
def scale_image(features):
    image = tf.cast(features['image'], tf.float32)
    image /= 255
    return image, features['label']

In [ ]:
train_batches = train_ds.map(scale_image).shuffle(1024).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

test_batches = test_ds.map(scale_image).shuffle(1024).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
# plotting helper function
def plothist(hist):
    
    plt.subplot(1, 2, 1)
    plt.plot(hist.history['accuracy'])
    plt.plot(hist.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='lower right')

    plt.subplot(1, 2, 2)
    plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper right')


    plt.show()

In [ ]:
!pip install git+https://github.com/qubvel/classification_models.git

  Cloning https://github.com/qubvel/classification_models.git to /tmp/pip-req-build-w22aapn2
  Running command git clone -q https://github.com/qubvel/classification_models.git /tmp/pip-req-build-w22aapn2
  Running command git submodule update --init --recursive -q
  Created wheel for image-classifiers: filename=image_classifiers-1.0.0-cp36-none-any.whl size=20030 sha256=cea3850873574356d38b9afde01194cde1252126b8df2cab4353fbb6543c3a21
  Stored in directory: /tmp/pip-ephem-wheel-cache-0y9pmo71/wheels/de/2b/fd/29a6d33edb8c28bc7d94e95ea1d39c9a218ac500a3cfb1b197
Successfully built image-classifiers


In [ ]:
from classification_models.tfkeras import Classifiers

In [ ]:
ResNet18, preprocess_input = Classifiers.get('resnet18')
resnet18 = ResNet18((224, 224, 3), weights='imagenet')
resnet18.summary() 

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 224, 224, 3)  9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_18 (ZeroPadding2 (None, 230, 230, 3)  0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 112, 112, 64) 9408        zero_padding2d_18[0][0]          
____________________________________________________________________________________________

In [ ]:
alpha = 0.003  # weight decay coefficient

for layer in resnet18.layers:
    if isinstance(layer, keras.layers.Conv2D) or isinstance(layer, keras.layers.Dense):
        layer.add_loss(lambda: keras.regularizers.l2(alpha)(layer.kernel))
    if hasattr(layer, 'bias_regularizer') and layer.use_bias:
        layer.add_loss(lambda: keras.regularizers.l2(alpha)(layer.bias))

In [ ]:
x = resnet18.layers[-2].output
output = Dense(units=10,activation='softmax')(x)
model = Model(inputs=[resnet18.input], outputs=[output])
#model.summary()

In [ ]:
for layer in model.layers[35:]:
    layer.trainable = True
for layer in model.layers[:35]:
    layer.trainable = False
for i, layer in enumerate(model.layers):
    print(i, layer.name, layer.trainable)

0 data False
1 bn_data False
2 zero_padding2d_18 False
3 conv0 False
4 bn0 False
5 relu0 False
6 zero_padding2d_19 False
7 pooling0 False
8 stage1_unit1_bn1 False
9 stage1_unit1_relu1 False
10 zero_padding2d_20 False
11 stage1_unit1_conv1 False
12 stage1_unit1_bn2 False
13 stage1_unit1_relu2 False
14 zero_padding2d_21 False
15 stage1_unit1_conv2 False
16 stage1_unit1_sc False
17 add_8 False
18 stage1_unit2_bn1 False
19 stage1_unit2_relu1 False
20 zero_padding2d_22 False
21 stage1_unit2_conv1 False
22 stage1_unit2_bn2 False
23 stage1_unit2_relu2 False
24 zero_padding2d_23 False
25 stage1_unit2_conv2 False
26 add_9 False
27 stage2_unit1_bn1 False
28 stage2_unit1_relu1 False
29 zero_padding2d_24 False
30 stage2_unit1_conv1 False
31 stage2_unit1_bn2 False
32 stage2_unit1_relu2 False
33 zero_padding2d_25 False
34 stage2_unit1_conv2 False
35 stage2_unit1_sc True
36 add_10 True
37 stage2_unit2_bn1 True
38 stage2_unit2_relu1 True
39 zero_padding2d_26 True
40 stage2_unit2_conv1 True
41 stage2_u

In [ ]:
newInput = Input(batch_shape=(None, 32, 32, 3))
resizedImg = Lambda(lambda image: ktf.image.resize(image, (224, 224)))(newInput)
newOutputs = model(resizedImg)
model = Model(newInput, newOutputs)

In [ ]:

STEPS_PER_EPOCH =len(X_train)//100

step = tf.Variable(0, trainable=False)
boundaries = [30*STEPS_PER_EPOCH,50*STEPS_PER_EPOCH,80*STEPS_PER_EPOCH]
values = [ 0.1, 0.01,0.001,0.0001]
learning_rate_fn = keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries, values)

In [ ]:
opt = SGD(learning_rate=learning_rate_fn)#,momentum=0.9,decay=0.01,nesterov=False
#opt = Adam(learning_rate=0.0001,beta_1=0.9,beta_2=0.99,epsilon=0.1,amsgrad=False,name="Adam",)
#model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
#model.summary()

In [ ]:
loss_avg = tf.keras.metrics.Mean(name='loss')
accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="acc")
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

class CustomModel(tf.keras.Model):
  def train_step(self,data):
    x_treain_batch,y_train_batch=data

    with tf.GradientTape() as tape:
      y_pred=self(x_treain_batch,training=True)

      loss= loss_fn(y_train_batch,y_pred)


    gradients=tape.gradient(loss,self.trainable_variables)

    self.optimizer.apply_gradients(zip(gradients,self.trainable_variables))

    loss_avg.update_state(loss)
    accuracy.update_state(y_train_batch,y_pred=y_pred)
    return {'loss': loss_avg.result(), 'accuracy': accuracy.result()}

In [ ]:
c_model = CustomModel(model.input, model.output)

In [ ]:
c_model.compile(optimizer=opt)

In [ ]:
history = c_model.fit(x=train_batches,
              epochs=5,
              validation_data=test_batches,
              shuffle=True)

Epoch 1/5
1563/1563 [==============================] - 113s 71ms/step - loss: 4.1169 - accuracy: 0.1106 - val_loss: 0.5914
Epoch 2/5
1563/1563 [==============================] - 111s 71ms/step - loss: 2.2921 - accuracy: 0.2193 - val_loss: 0.6422
Epoch 3/5
1563/1563 [==============================] - 112s 71ms/step - loss: 1.8426 - accuracy: 0.3631 - val_loss: 0.7724
Epoch 4/5
1563/1563 [==============================] - 111s 71ms/step - loss: 1.5445 - accuracy: 0.4656 - val_loss: 0.9201
Epoch 5/5
1563/1563 [==============================] - 111s 71ms/step - loss: 1.3345 - accuracy: 0.5386 - val_loss: 1.0671


In [ ]:
num_classes=10
y_pred = model.predict(x=test_batches)
labels1 = np.arange(num_classes)
print('Auc = %f' %  (roc_auc_score(y=test_batches, y_pred=y_pred.round(),average='macro',multi_class='ovo',labels=labels1)))
print('f1_score = %f' % (f1_score(y=test_batches, y_pred=y_pred.round(), average="macro")))
print('precision = %f' % (precision_score(y=test_batches, y_pred=y_pred.round(), average="macro")))
print('recall  = %f' % (recall_score(y=test_batches, y_pred=y_pred.round(), average="macro")))
print('accuracy  = %f' % (accuracy_score(y=test_batches, y_pred=y_pred.round()))) 

TypeError: ignored

In [ ]:
plothist(history)